In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import time 
from tensorflow.keras.models import Sequential, load_model
from keras.layers import SimpleRNN, Dense
import tensorflow as tf
%load_ext tensorboard

In [2]:
data_path_and_labels = pd.read_csv("../mapped_data/camel-1.4.csv")
data_path_and_labels['data_path'][0]

'dataset/promise/defect data/camel-1.4/camel-camel-1.4.0/components/camel-irc/src/main/java/org/apache/camel/component/irc/IrcExchange.java.restrictedcontent.embed'

In [3]:
data_path_and_labels['data_path'] = '../' + data_path_and_labels['data_path']
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/camel-1.4/camel-camel-1.4.0/components/camel-irc/src/main/java/org/apache/camel/component/irc/IrcExchange.java.restrictedcontent.embed'

In [4]:
data_path_and_labels['data_path'] = data_path_and_labels['data_path'].str.replace('.embed','')
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/camel-1.4/camel-camel-1.4.0/components/camel-irc/src/main/java/org/apache/camel/component/irc/IrcExchange.java.restrictedcontent'

In [5]:
words = []
for index, row in data_path_and_labels.iterrows():
    with open(row['data_path']) as f:
        lines = f.read().splitlines()
    words.append(lines)

In [6]:
d = pd.read_csv("../mapped_data/camel-1.4.csv")
d['data_path'][0]

'dataset/promise/defect data/camel-1.4/camel-camel-1.4.0/components/camel-irc/src/main/java/org/apache/camel/component/irc/IrcExchange.java.restrictedcontent.embed'

In [7]:
d['data_path'] = '../' + d['data_path']
d['data_path'][0]

'../dataset/promise/defect data/camel-1.4/camel-camel-1.4.0/components/camel-irc/src/main/java/org/apache/camel/component/irc/IrcExchange.java.restrictedcontent.embed'

In [8]:
d

,data_path,bugs
0,../dataset/promise/defect data/camel-1.4/camel...,0
1,../dataset/promise/defect data/camel-1.4/camel...,0
2,../dataset/promise/defect data/camel-1.4/camel...,0
3,../dataset/promise/defect data/camel-1.4/camel...,0
4,../dataset/promise/defect data/camel-1.4/camel...,0
...,...,...
842,../dataset/promise/defect data/camel-1.4/camel...,0
843,../dataset/promise/defect data/camel-1.4/camel...,1
844,../dataset/promise/defect data/camel-1.4/camel...,0
845,../dataset/promise/defect data/camel-1.4/camel...,0


In [9]:
train, test = train_test_split(d, test_size=0.2)

In [10]:
test_x = []
for index, row in test.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    test_x.append(lines)

In [11]:
train_x = []
for index, row in train.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    train_x.append(lines)

In [12]:
vocab = float(max(np.max(train_x), np.max(test_x)))
vocab

7766.0

In [13]:
train_y = train[' bugs'].to_numpy()
test_y = test[' bugs'].to_numpy()

In [14]:
train_y

array([ 0,  0, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,
        0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  0,
        0,  0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,
        0,  2,  0,  0,  2,  0,  1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  3,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  1,  0,  0,  0,  1,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  3,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  3,  0,
        0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  3,
        0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  7,  0,  0,  2,  0,  0,  0,
        0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  1,
        0,  0,  1,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  2,  0

In [15]:
train_x = np.array(train_x)
test_x = np.array(test_x)

In [16]:
train_x.shape

(677, 2094)

In [17]:
# define the model
model = Sequential()
model.add(tf.keras.layers.Embedding(10671, 30, input_length=train_x.shape[1]))
model.add(tf.keras.layers.Flatten())
model.add(Dense(1, activation='relu'))
# compile the model
model.compile(loss = 'mse' , optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [18]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, callbacks=[tensorboard_callback])

Epoch 1/100
22/22 [==============================] - 1s 10ms/step - loss: 1.7495 - mse: 1.7495 - mae: 0.5916 - root_mean_squared_error: 1.3227 - mean_squared_logarithmic_error: 0.2210
Epoch 2/100
22/22 [==============================] - 0s 9ms/step - loss: 1.6566 - mse: 1.6566 - mae: 0.6501 - root_mean_squared_error: 1.2871 - mean_squared_logarithmic_error: 0.2189
Epoch 3/100
22/22 [==============================] - 0s 9ms/step - loss: 1.6093 - mse: 1.6093 - mae: 0.6155 - root_mean_squared_error: 1.2686 - mean_squared_logarithmic_error: 0.2051
Epoch 4/100
22/22 [==============================] - 0s 9ms/step - loss: 1.5411 - mse: 1.5411 - mae: 0.5392 - root_mean_squared_error: 1.2414 - mean_squared_logarithmic_error: 0.1750
Epoch 5/100
22/22 [==============================] - 0s 10ms/step - loss: 1.4764 - mse: 1.4764 - mae: 0.6109 - root_mean_squared_error: 1.2151 - mean_squared_logarithmic_error: 0.1866
Epoch 6/100
22/22 [==============================] - 0s 9ms/step - loss: 1.4444 - m

22/22 [==============================] - 0s 9ms/step - loss: 0.3132 - mse: 0.3132 - mae: 0.2660 - root_mean_squared_error: 0.5596 - mean_squared_logarithmic_error: 0.0602
Epoch 45/100
22/22 [==============================] - 0s 9ms/step - loss: 0.3046 - mse: 0.3046 - mae: 0.1939 - root_mean_squared_error: 0.5519 - mean_squared_logarithmic_error: 0.0444
Epoch 46/100
22/22 [==============================] - 0s 9ms/step - loss: 0.2890 - mse: 0.2890 - mae: 0.2494 - root_mean_squared_error: 0.5376 - mean_squared_logarithmic_error: 0.0553
Epoch 47/100
22/22 [==============================] - 0s 10ms/step - loss: 0.2795 - mse: 0.2795 - mae: 0.2139 - root_mean_squared_error: 0.5287 - mean_squared_logarithmic_error: 0.0462
Epoch 48/100
22/22 [==============================] - 0s 10ms/step - loss: 0.2622 - mse: 0.2622 - mae: 0.1805 - root_mean_squared_error: 0.5121 - mean_squared_logarithmic_error: 0.0383
Epoch 49/100
22/22 [==============================] - 0s 9ms/step - loss: 0.2609 - mse: 0.2

22/22 [==============================] - 0s 9ms/step - loss: 0.0454 - mse: 0.0454 - mae: 0.0744 - root_mean_squared_error: 0.2131 - mean_squared_logarithmic_error: 0.0121
Epoch 89/100
22/22 [==============================] - 0s 10ms/step - loss: 0.0436 - mse: 0.0436 - mae: 0.0715 - root_mean_squared_error: 0.2089 - mean_squared_logarithmic_error: 0.0119
Epoch 90/100
22/22 [==============================] - 0s 9ms/step - loss: 0.0422 - mse: 0.0422 - mae: 0.0681 - root_mean_squared_error: 0.2055 - mean_squared_logarithmic_error: 0.0113
Epoch 91/100
22/22 [==============================] - 0s 9ms/step - loss: 0.0405 - mse: 0.0405 - mae: 0.0667 - root_mean_squared_error: 0.2013 - mean_squared_logarithmic_error: 0.0112
Epoch 92/100
22/22 [==============================] - 0s 9ms/step - loss: 0.0397 - mse: 0.0397 - mae: 0.0672 - root_mean_squared_error: 0.1993 - mean_squared_logarithmic_error: 0.0113
Epoch 93/100
22/22 [==============================] - 0s 9ms/step - loss: 0.0379 - mse: 0.03

In [19]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6006 (pid 21467), started 0:31:40 ago. (Use '!kill 21467' to kill it.)

In [20]:
score = model.evaluate(test_x, test_y)
score

6/6 [==============================] - 0s 3ms/step - loss: 0.8535 - mse: 0.8535 - mae: 0.5142 - root_mean_squared_error: 0.9239 - mean_squared_logarithmic_error: 0.1764


[0.853528618812561,
 0.853528618812561,
 0.5142396092414856,
 0.9238661527633667,
 0.1763971894979477]

In [21]:
data_path_and_labels = pd.read_csv("../mapped_data/camel-1.4.csv")
data_path_and_labels['data_path'][0]

'dataset/promise/defect data/camel-1.4/camel-camel-1.4.0/components/camel-irc/src/main/java/org/apache/camel/component/irc/IrcExchange.java.restrictedcontent.embed'

In [22]:
data_path_and_labels['data_path'] = '../' + data_path_and_labels['data_path']
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/camel-1.4/camel-camel-1.4.0/components/camel-irc/src/main/java/org/apache/camel/component/irc/IrcExchange.java.restrictedcontent.embed'

In [23]:
embeddings = []
for index, row in data_path_and_labels.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    embeddings.append(lines)

In [24]:
df = pd.DataFrame(embeddings)
df

,0,1,2,3,4,5,6,7,8,9,...,2084,2085,2086,2087,2088,2089,2090,2091,2092,2093
0,990.0,984.0,2612.0,990.0,2986.0,252.0,5920.0,700.0,2612.0,984.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,992.0,1971.0,220.0,1971.0,1154.0,1115.0,4378.0,992.0,911.0,2937.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,986.0,1971.0,7274.0,1971.0,4872.0,1971.0,5917.0,1971.0,5659.0,1971.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,991.0,1971.0,5509.0,1971.0,7274.0,915.0,7652.0,1971.0,7714.0,1971.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,984.0,3764.0,1411.0,3669.0,990.0,5486.0,991.0,1971.0,7575.0,4422.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,1638.0,726.0,1186.0,1451.0,1971.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
843,159.0,3226.0,1529.0,694.0,3748.0,1971.0,6396.0,7501.0,3139.0,3748.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
844,1747.0,694.0,3669.0,1971.0,2584.0,1971.0,5516.0,1747.0,3669.0,694.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
845,152.0,1971.0,2593.0,1971.0,5516.0,152.0,2593.0,1971.0,5516.0,1971.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
data = pd.concat([data_path_and_labels, df], axis = 1)
data = data.drop('data_path', 1)
data

,bugs,0,1,2,3,4,5,6,7,8,...,2084,2085,2086,2087,2088,2089,2090,2091,2092,2093
0,0,990.0,984.0,2612.0,990.0,2986.0,252.0,5920.0,700.0,2612.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,992.0,1971.0,220.0,1971.0,1154.0,1115.0,4378.0,992.0,911.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,986.0,1971.0,7274.0,1971.0,4872.0,1971.0,5917.0,1971.0,5659.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,991.0,1971.0,5509.0,1971.0,7274.0,915.0,7652.0,1971.0,7714.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,984.0,3764.0,1411.0,3669.0,990.0,5486.0,991.0,1971.0,7575.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,0,1638.0,726.0,1186.0,1451.0,1971.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
843,1,159.0,3226.0,1529.0,694.0,3748.0,1971.0,6396.0,7501.0,3139.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
844,0,1747.0,694.0,3669.0,1971.0,2584.0,1971.0,5516.0,1747.0,3669.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
845,0,152.0,1971.0,2593.0,1971.0,5516.0,152.0,2593.0,1971.0,5516.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
train, test = train_test_split(data, test_size=0.2)

In [27]:
train

,bugs,0,1,2,3,4,5,6,7,8,...,2084,2085,2086,2087,2088,2089,2090,2091,2092,2093
541,0,1423.0,1423.0,1423.0,3748.0,1971.0,3748.0,4358.0,1971.0,6396.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
268,0,578.0,3418.0,3597.0,661.0,3596.0,1411.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
786,0,3123.0,646.0,694.0,7622.0,1971.0,6238.0,1971.0,5877.0,1212.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62,0,2234.0,1154.0,1115.0,4378.0,2234.0,2230.0,3571.0,1971.0,5912.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
169,0,1962.0,1154.0,1115.0,4378.0,1962.0,1971.0,2012.0,1971.0,903.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,0,1507.0,1154.0,1115.0,4378.0,1507.0,1508.0,6327.0,565.0,2700.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
184,0,432.0,1971.0,452.0,694.0,3669.0,432.0,2986.0,252.0,5920.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
427,0,1638.0,1451.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
164,0,952.0,726.0,951.0,5696.0,952.0,952.0,5596.0,1971.0,5696.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
test

,bugs,0,1,2,3,4,5,6,7,8,...,2084,2085,2086,2087,2088,2089,2090,2091,2092,2093
163,0,1248.0,726.0,1134.0,849.0,4858.0,73.0,849.0,1134.0,2975.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
439,1,670.0,3025.0,670.0,3129.0,1544.0,6416.0,1734.0,6082.0,1544.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
198,0,1971.0,1186.0,1971.0,1768.0,1971.0,452.0,1971.0,1773.0,1971.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,0,47.0,1154.0,1115.0,4378.0,47.0,4982.0,3725.0,1543.0,6074.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
142,0,1374.0,127.0,6082.0,1374.0,6082.0,127.0,3669.0,694.0,4089.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,1,2094.0,726.0,2094.0,2094.0,3748.0,708.0,3748.0,7515.0,1971.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
231,0,1396.0,2829.0,2829.0,5486.0,1223.0,1443.0,5837.0,4077.0,1443.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
463,0,2212.0,1583.0,6389.0,3226.0,1529.0,694.0,3748.0,1971.0,2206.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
128,0,2115.0,2115.0,7622.0,1971.0,2877.0,2114.0,7622.0,2877.0,2115.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
Y_train = train[' bugs']
Y_test = test[' bugs']

In [30]:
train_y = Y_train.to_numpy()
test_y = Y_test.to_numpy()

In [31]:
train = train.drop(' bugs', axis=1)
test = test.drop(' bugs', axis=1)

In [32]:
X_train = train.values
X_test = test.values

In [33]:
train_x = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
test_x = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [34]:
# Designing and initializing the model.
model = Sequential()
model.add(SimpleRNN(100, input_shape = (1,20), dropout = 0.2, return_sequences=True))
model.add(SimpleRNN(80, dropout = 0.2, return_sequences=True))
model.add(SimpleRNN(60, dropout = 0.2, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = 'adam' , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [35]:
# Fitting the model on training data.
history = model.fit(train_x, train_y, epochs = 100)

Epoch 1/100
22/22 [==============================] - 1s 3ms/step - loss: 1.8699 - mse: 1.8699 - mae: 0.4442 - root_mean_squared_error: 1.3674 - mean_squared_logarithmic_error: 0.2462
Epoch 2/100
22/22 [==============================] - 0s 3ms/step - loss: 1.7961 - mse: 1.7961 - mae: 0.4079 - root_mean_squared_error: 1.3402 - mean_squared_logarithmic_error: 0.2245
Epoch 3/100
22/22 [==============================] - 0s 3ms/step - loss: 1.6473 - mse: 1.6473 - mae: 0.5073 - root_mean_squared_error: 1.2835 - mean_squared_logarithmic_error: 0.2310
Epoch 4/100
22/22 [==============================] - 0s 3ms/step - loss: 1.5767 - mse: 1.5767 - mae: 0.4157 - root_mean_squared_error: 1.2557 - mean_squared_logarithmic_error: 0.2006
Epoch 5/100
22/22 [==============================] - 0s 3ms/step - loss: 1.6606 - mse: 1.6606 - mae: 0.4611 - root_mean_squared_error: 1.2887 - mean_squared_logarithmic_error: 0.2228
Epoch 6/100
22/22 [==============================] - 0s 3ms/step - loss: 1.5280 - mse

22/22 [==============================] - 0s 4ms/step - loss: 1.5418 - mse: 1.5418 - mae: 0.5431 - root_mean_squared_error: 1.2417 - mean_squared_logarithmic_error: 0.2296
Epoch 46/100
22/22 [==============================] - 0s 3ms/step - loss: 1.3654 - mse: 1.3654 - mae: 0.4751 - root_mean_squared_error: 1.1685 - mean_squared_logarithmic_error: 0.1978
Epoch 47/100
22/22 [==============================] - 0s 3ms/step - loss: 1.3840 - mse: 1.3840 - mae: 0.4658 - root_mean_squared_error: 1.1764 - mean_squared_logarithmic_error: 0.1926
Epoch 48/100
22/22 [==============================] - 0s 3ms/step - loss: 1.3760 - mse: 1.3760 - mae: 0.4798 - root_mean_squared_error: 1.1730 - mean_squared_logarithmic_error: 0.2024
Epoch 49/100
22/22 [==============================] - 0s 3ms/step - loss: 1.4170 - mse: 1.4170 - mae: 0.5204 - root_mean_squared_error: 1.1904 - mean_squared_logarithmic_error: 0.2193
Epoch 50/100
22/22 [==============================] - 0s 3ms/step - loss: 1.3877 - mse: 1.387

22/22 [==============================] - 0s 3ms/step - loss: 1.2681 - mse: 1.2681 - mae: 0.5766 - root_mean_squared_error: 1.1261 - mean_squared_logarithmic_error: 0.2042
Epoch 90/100
22/22 [==============================] - 0s 3ms/step - loss: 1.5646 - mse: 1.5646 - mae: 0.5461 - root_mean_squared_error: 1.2508 - mean_squared_logarithmic_error: 0.1992
Epoch 91/100
22/22 [==============================] - 0s 3ms/step - loss: 1.4237 - mse: 1.4237 - mae: 0.5383 - root_mean_squared_error: 1.1932 - mean_squared_logarithmic_error: 0.1917
Epoch 92/100
22/22 [==============================] - 0s 3ms/step - loss: 1.2687 - mse: 1.2687 - mae: 0.5370 - root_mean_squared_error: 1.1264 - mean_squared_logarithmic_error: 0.1992
Epoch 93/100
22/22 [==============================] - 0s 3ms/step - loss: 1.2741 - mse: 1.2741 - mae: 0.6073 - root_mean_squared_error: 1.1288 - mean_squared_logarithmic_error: 0.2165
Epoch 94/100
22/22 [==============================] - 0s 3ms/step - loss: 1.2423 - mse: 1.242

In [36]:
score = model.evaluate(test_x, test_y)
score

6/6 [==============================] - 0s 1ms/step - loss: 1.1889 - mse: 1.1889 - mae: 0.4939 - root_mean_squared_error: 1.0903 - mean_squared_logarithmic_error: 0.1669


[1.1888604164123535,
 1.1888604164123535,
 0.4938718378543854,
 1.090348720550537,
 0.16694718599319458]